In [ ]:
from huggingface_hub import login

# Yêu cầu nhập token từ người dùng
token = input("Nhập token Hugging Face của bạn: ")

# Thực hiện đăng nhập
login(token)

In [2]:
import pandas as pd

df = pd.read_csv('/kaggle/input/task2-ielts-dataset/train.csv')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
import torch
import sys

def extract_band_scores(text):
    # Danh sách các tiêu chí cần trích xuất
    criteria = [
        "Task Achievement",
        "Coherence and Cohesion",
        "Lexical Resource",
        "Grammatical Range and Accuracy",
        "Overall Band Score"
    ]

    band_scores = {}

    for criterion in criteria:
        # Regex sửa lại để bắt cả số nguyên (X) và số thập phân (X.Y)
        pattern = re.compile(rf'{re.escape(criterion)}.*?<?(\d+(?:\.\d)?)', re.DOTALL)
        match = pattern.search(text)
        if match:
            band_scores[criterion] = match.group(1)

    return band_scores

# Kiểm tra GPU
assert torch.cuda.is_available(), "Không tìm thấy GPU!"
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tải model với cấu hình tối ưu
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

def query_llm(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        pad_token_id=tokenizer.eos_token_id  # Thêm để tránh warning
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

responses = []

# i = 0
prompt = df['evaluation'][0] + """

        Extract only the citeration and band scores like:
        Citeration: Score.
        """
response = query_llm(prompt)
temp = extract_band_scores(response)
df_merged = pd.DataFrame([temp]).astype(float)
print(df_merged)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


   Task Achievement  Coherence and Cohesion  Lexical Resource  \
0               8.0                     8.5               7.5   

   Grammatical Range and Accuracy  Overall Band Score  
0                             8.0                 8.5  


In [ ]:
for i in range(1,len(df)):
    try:
        prompt = df['evaluation'][i] + """

        Extract only the citeration and band scores like:
        Citeration: Score.
        """
        response = query_llm(prompt)

        temp = extract_band_scores(response)
        check1 = False

        required_columns = [
            "Task Achievement",
            "Coherence and Cohesion",
            "Lexical Resource",
            "Grammatical Range and Accuracy",
            "Overall Band Score"
        ]

        # Kiểm tra và bổ sung các cột bị thiếu
        for col in required_columns:
            if col not in temp:
                check1 = True
                temp[col] = '-1'

        df_temp = pd.DataFrame([temp]).astype(float)

        df_merged = pd.concat([df_merged, df_temp], ignore_index=True)

        if check1 == True:
           sys.stdout.write("Dữ liệu thiếu xảy ra ở " + str(i) + "\n")

        #sys.stdout.write(str(i) + "\n")
    except Exception as e:
         sys.stdout.write("Có lỗi ở phần tử " + str(i) + "\n")
         continue

In [ ]:
#df2.to_csv('/kaggle/working/processed_dataset_0_3999.csv',index=False)
df_merged.to_csv('/kaggle/working/cleaned_0_9832.csv',index=False)